# IBM Applied Data Science Capstone Project

## Project1
Build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

To create the dataframe:

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.


2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.


6. Use the .shape method to print the number of rows of your dataframe.

## Project2 
Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

We will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code.

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

## Project3
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto

In [1]:
#Import Libraries

import numpy as np
import pandas as pd

import json
import urllib.request

import matplotlib.cm as cm
import matplotlib.colors as colors
import requests

#!conda install -c conda-forge geopy --yes

#Install and import
!conda install -c conda-forge folium=0.5.0 --yes
import folium

import bs4 as bs
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans


Solving environment: done

# All requested packages already installed.



### Scrap data from Wikipedia page into a DataFrame

In [2]:

import geopy

# send the GET request
#data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(source,'html.parser')

#
table = soup.find('table')
table_rows = table.find_all('tr')

# create lists to store data
list_data = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        list_data.append(row)

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood


In [3]:
df = pd.DataFrame(list_data, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


In [4]:
# Cells that have an assigned borough. 
#And Ignore cells with a borough that is 'Not assigned'.

df_dropna = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df_dropna.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### More than one neighborhood can exist in one postal code area. 
### For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [5]:
# More than one neighborhood can exist in one postal code area.

df_grouped = df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.head()


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [6]:
#Group
df_grouped.loc[df_grouped['Neighbourhood']=='Not assigned', ['Neighbourhood']]
df_grouped.head()


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Show the data

In [7]:
# Represent the data
df1 = pd.DataFrame(columns=["PostalCode", "Borough", "Neighbourhood"])

test_data_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_data_list:
    df1 = df1.append(df_grouped[df_grouped["PostalCode"]==postcode], ignore_index=True)
    
df1

,PostalCode,Borough,Neighbourhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,Parkview Hill / Woodbine Gardens
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,Wexford / Maryvale
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


 
 ### Use the .shape method to print the number of rows of your dataframe.
 

In [8]:
# Print the number of rows of the cleaned dataframe
df_grouped.shape


(103, 3)

### Getting the latitude and the longitude coordinates of each neighborhood.
The geopy/geocoder package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [9]:
# Retrieve coordinates from the csv file on
coordinates = pd.read_csv("http://cocl.us/Geospatial_data")
coordinates.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# In csv file one column name is different from the other file
#So replace or rename it
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

# Combine both the tables
df_new = df_grouped.merge(coordinates, on="PostalCode", how="left")
df_new.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Represent the data according to question

In [11]:
# Represent the data
df1 = pd.DataFrame(columns=["PostalCode", "Borough", "Neighbourhood","Latitude", "Longitude"])

test_data_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_data_list:
    df1 = df1.append(df_new[df_new["PostalCode"]==postcode], ignore_index=True)
    
df1

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,Wexford / Maryvale,43.750072,-79.295849
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420


## The latitude and longitude values of Toronto

In [12]:
location = 'Toronto'

geolocator = Nominatim(user_agent="loc")
location = geolocator.geocode(location)
latitude = location.latitude
longitude = location.longitude
print('Coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Coordinate of Toronto are 43.6534817, -79.3839347.


## Create map of Toronto using latitude and longitude values
### Install and import folium

In [13]:
#Install and import
#!conda install -c conda-forge folium=0.5.0 --yes
#import folium

 ### Create a map of Toronto with neighborhoods

In [14]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=8)

# add markers to map
for lat, long, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

In [15]:
# just focus on borough
data1 = df_new[df_new['Borough'] == 'Scarborough'].reset_index(drop=True)
data1.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
location1 = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="loc1")
location = geolocator.geocode(location1)
latitude = location.latitude
longitude = location.longitude
print('Coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

Coordinate of Scarborough are 43.773077, -79.257774.


In [17]:
# Create map of Scarborough, Toronto
# Using latitude and longitude values
map_new = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, long, label in zip(data1['Latitude'], data1['Longitude'], data1['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_new)  
    
map_new

### Use the Foursquare API to explore the neighborhoods of the Toronto

In [18]:
# Define Foursquare Credentials and Version
# Foursquare client id and client secret
CLIENT_ID = 'YURAW5R5XNEOB3HBPIL1TKN4MB2HM2VBXGPFEZATUZ2K23K1'
CLIENT_SECRET = 'BG0ZCMUVHPLYEZZCOF4A3YBG0HERLP5UYLOJDJNUZRZA2SBZ'

#Version i.e date
VERSION = '20201404'

### Set the limit and radius to explore the neighbourhoods

In [19]:
#Explore the neighbourhoods

# limit of number of venues returned by Foursquare API
LIMIT = 100
# define radius
radius = 500 

# create URL
for lat, long, post, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['PostalCode'], df_new['Borough'], df_new['Neighbourhood']):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        long, 
        radius, 
        LIMIT)
# display URL
print("URL -",url) 

URL - https://api.foursquare.com/v2/venues/explore?&client_id=YURAW5R5XNEOB3HBPIL1TKN4MB2HM2VBXGPFEZATUZ2K23K1&client_secret=BG0ZCMUVHPLYEZZCOF4A3YBG0HERLP5UYLOJDJNUZRZA2SBZ&v=20201404&ll=43.706748299999994,-79.5940544&radius=500&limit=100


In [20]:
#See in JSON format
results = requests.get(url).json()
#results = requests.get(url).json()["response"]['groups'][0]['items']
print ("JSON Format")
results

JSON Format


{'meta': {'code': 200, 'requestId': '5e95d29b60ba08001b4fecab'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.711248304499996,
    'lng': -79.58784096674265},
   'sw': {'lat': 43.70224829549999, 'lng': -79.60026783325736}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5273f6f311d2137549494e29',
       'name': 'Economy Rent A Car',
       'location': {'lat': 43.708471,
        'lng': -79.589943,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.708471,
          'lng': -79.589943}],


In [21]:
'''venues = []
for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue["venue"]["name"], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))'''


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:

''''# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()'''

# clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

nearby_venues.head()


2 venues were returned by Foursquare.


,name,categories,lat,lng
0,Economy Rent A Car,Rental Car Location,43.708471,-79.589943
1,Saand Rexdale,Drugstore,43.705072,-79.598725


In [23]:
# Explore Neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, long in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
# get scarborough venues
scarborough_venues = getNearbyVenues(names=data1['Neighbourhood'],
                                   latitudes=data1['Latitude'],
                                   longitudes=data1['Longitude']
                                  )

print("\n\nTotal Venues:\n",scarborough_venues.shape)
scarborough_venues.head(10)

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge


Total Venues:
 (94, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Malvern / Rouge,43.806686,-79.194353,T Hamilton & Son Roofing Inc,43.807985,-79.198194,Construction & Landscaping
2,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,Guildwood / Morningside / West Hill,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,Guildwood / Morningside / West Hill,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,Guildwood / Morningside / West Hill,43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
7,Guildwood / Morningside / West Hill,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
8,Guildwood / Morningside / West Hill,43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
9,Guildwood / Morningside / West Hill,43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection


In [25]:
scarborough_venues.groupby(["Neighbourhood"]).count()
#print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))


,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
Birch Cliff / Cliffside West,4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
Clarks Corners / Tam O'Shanter / Sullivan,14,14,14,14,14,14
Cliffside / Cliffcrest / Scarborough Village West,2,2,2,2,2,2
Dorset Park / Wexford Heights / Scarborough Town Centre,6,6,6,6,6,6
Golden Mile / Clairlea / Oakridge,10,10,10,10,10,10
Guildwood / Morningside / West Hill,7,7,7,7,7,7
Kennedy Park / Ionview / East Birchmount Park,5,5,5,5,5,5


In [26]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))


There are 56 uniques categories.


In [27]:

# analyze each neighborhood

# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighbourhood'] = scarborough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

print("\nAfter Analyzing:",scarborough_onehot.shape)
scarborough_onehot.head()


After Analyzing: (94, 57)


,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,Bus Line,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Malvern / Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Malvern / Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rouge Hill / Port Union / Highland Creek,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rouge Hill / Port Union / Highland Creek,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Guildwood / Morningside / West Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
scarborough_grouped = scarborough_onehot.groupby('Neighbourhood').mean().reset_index()

print("\n",scarborough_grouped.shape)
scarborough_grouped



 (16, 57)


,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,Bus Line,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.200000,0.0000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.20,0.0,0.0000,0.000000,0.000000
1,Birch Cliff / Cliffside West,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.0,0.0000,0.000000,0.000000
2,Cedarbrae,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.0000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0000,0.125000,0.000000
3,Clarks Corners / Tam O'Shanter / Sullivan,0.0,0.000,0.000000,0.000000,0.071429,0.0,0.000000,0.0000,0.0,...,0.142857,0.0,0.071429,0.000000,0.071429,0.00,0.0,0.0000,0.071429,0.000000
4,Cliffside / Cliffcrest / Scarborough Village West,0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0000,0.000000,0.000000
5,Dorset Park / Wexford Heights / Scarborough To...,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0000,0.000000,0.166667
6,Golden Mile / Clairlea / Oakridge,0.0,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.0000,0.2,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.1,0.0000,0.000000,0.000000
7,Guildwood / Morningside / West Hill,0.0,0.000,0.000000,0.000000,0.142857,0.0,0.142857,0.0000,0.0,...,0.000000,0.0,0.142857,0.000000,0.000000,0.00,0.0,0.0000,0.000000,0.000000
8,Kennedy Park / Ionview / East Birchmount Park,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0000,0.000000,0.000000
9,Malvern / Rouge,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0000,0.000000,0.000000


In [29]:

# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1               Skating Rink   0.2
2             Breakfast Spot   0.2
3                     Lounge   0.2
4             Clothing Store   0.2


----Birch Cliff / Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                  venue  freq
0      Hakka Restaurant  0.12
1  Caribbean Restaurant  0.12
2       Thai Restaurant  0.12
3                Bakery  0.12
4                  Bank  0.12


----Clarks Corners / Tam O'Shanter / Sullivan----
                 venue  freq
0          Pizza Place  0.14
1         Intersection  0.07
2  Rental Car Location  0.07
3  Fried Chicken Joint  0.07
4          Gas Station  0.07


----Cliffside / Cliffcrest / Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5
1  

In [30]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
# create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = scarborough_grouped['Neighbourhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,Birch Cliff / Cliffside West,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Cedarbrae,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Construction & Landscaping,Grocery Store
3,Clarks Corners / Tam O'Shanter / Sullivan,Pizza Place,Fried Chicken Joint,Chinese Restaurant,Gas Station,Italian Restaurant,Noodle House,Intersection,Pharmacy,Rental Car Location,Bank
4,Cliffside / Cliffcrest / Scarborough Village West,American Restaurant,Motel,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
5,Dorset Park / Wexford Heights / Scarborough To...,Indian Restaurant,Vietnamese Restaurant,Pet Store,Light Rail Station,Chinese Restaurant,Bank,Bar,Gas Station,Athletics & Sports,Fried Chicken Joint
6,Golden Mile / Clairlea / Oakridge,Bakery,Bus Line,Park,Soccer Field,Ice Cream Shop,Intersection,Metro Station,Bus Station,Vietnamese Restaurant,Construction & Landscaping
7,Guildwood / Morningside / West Hill,Mexican Restaurant,Bank,Intersection,Rental Car Location,Breakfast Spot,Electronics Store,Medical Center,Vietnamese Restaurant,Gas Station,Fried Chicken Joint
8,Kennedy Park / Ionview / East Birchmount Park,Coffee Shop,Discount Store,Department Store,Bus Station,Convenience Store,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
9,Malvern / Rouge,Fast Food Restaurant,Construction & Landscaping,Vietnamese Restaurant,Thai Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store


In [32]:

# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 4, 1, 1, 1, 1, 0], dtype=int32)

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = data1

#scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge scarborough_grouped with scarborough to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

# check the last columns!
scarborough_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,0.0,Fast Food Restaurant,Construction & Landscaping,Vietnamese Restaurant,Thai Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,2.0,History Museum,Bar,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,1.0,Mexican Restaurant,Bank,Intersection,Rental Car Location,Breakfast Spot,Electronics Store,Medical Center,Vietnamese Restaurant,Gas Station,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Vietnamese Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Construction & Landscaping,Grocery Store


In [75]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [76]:
# Cluster 1 
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0.0,Fast Food Restaurant,Construction & Landscaping,Vietnamese Restaurant,Thai Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store


In [77]:
# Cluster 2
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1.0,Mexican Restaurant,Bank,Intersection,Rental Car Location,Breakfast Spot,Electronics Store,Medical Center,Vietnamese Restaurant,Gas Station,Fried Chicken Joint
3,Scarborough,1.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Vietnamese Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,Scarborough,1.0,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Construction & Landscaping,Grocery Store
6,Scarborough,1.0,Coffee Shop,Discount Store,Department Store,Bus Station,Convenience Store,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
7,Scarborough,1.0,Bakery,Bus Line,Park,Soccer Field,Ice Cream Shop,Intersection,Metro Station,Bus Station,Vietnamese Restaurant,Construction & Landscaping
9,Scarborough,1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
10,Scarborough,1.0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Light Rail Station,Chinese Restaurant,Bank,Bar,Gas Station,Athletics & Sports,Fried Chicken Joint
11,Scarborough,1.0,Middle Eastern Restaurant,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Discount Store,College Stadium,Construction & Landscaping,Convenience Store
12,Scarborough,1.0,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,1.0,Pizza Place,Fried Chicken Joint,Chinese Restaurant,Gas Station,Italian Restaurant,Noodle House,Intersection,Pharmacy,Rental Car Location,Bank


In [82]:
# Cluster 3
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2.0,History Museum,Bar,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [83]:
# Cluster 4
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,3.0,Playground,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
14,Scarborough,3.0,Playground,Park,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [84]:
# Cluster 5
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,4.0,American Restaurant,Motel,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
